In [1]:
# from pyspark import SparkSession as sp
import pyspark as sp
from pydataset import data
from vega_datasets import data as vdata
import pyspark.sql.functions as sqlFunc

import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

**Create a spark session**

In [2]:
sp = sp.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/23 12:51:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
# create a dataframe
languages = ["Python", "Java", "C++", "JavaScript", "C#", "Ruby", "Swift", "Go", "Rust", "Kotlin", "PHP", "Perl", "Haskell", "Scala", "Lua", "TypeScript", "R", "Dart", "Elixir", "Julia"]
# create a pandas df
df = pd.DataFrame(languages, columns=["languages"])
# convert pandas dataframe to spark dataframe
sdf = sp.createDataFrame(df)

In [4]:
# show the shape of the dataframe
sdf.count(),len(sdf.columns)

(20, 1)

In [5]:
# show dataframe discription
sdf.describe().show()

+-------+----------+
|summary| languages|
+-------+----------+
|  count|        20|
|   mean|      null|
| stddev|      null|
|    min|        C#|
|    max|TypeScript|
+-------+----------+



In [6]:
# show the spark dataframe head
sdf.show(5)

+----------+
| languages|
+----------+
|    Python|
|      Java|
|       C++|
|JavaScript|
|        C#|
+----------+
only showing top 5 rows



### Load the mpg dataset as a spark dataframe.
- Create 1 column of output that contains a message like the one below:

```The 1999 audi a4 has a 4 cylinder engine.```
For each vehicle.

- Transform the trans column so that it only contains either manual or auto.

In [7]:
# load mpg data
mpg = data("mpg")
# Converto to spark
mpg = sp.createDataFrame(mpg)

In [8]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
# The 1999 audi a4 has a 4 cylinder engine. For each vehicle.
mpg.select(mpg.year, mpg.manufacturer, mpg.model, mpg.cyl).show(5)

+----+------------+-----+---+
|year|manufacturer|model|cyl|
+----+------------+-----+---+
|1999|        audi|   a4|  4|
|1999|        audi|   a4|  4|
|2008|        audi|   a4|  4|
|2008|        audi|   a4|  4|
|1999|        audi|   a4|  6|
+----+------------+-----+---+
only showing top 5 rows



In [10]:
# get value counts
mpg.groupby("trans").count().show()

+----------+-----+
|     trans|count|
+----------+-----+
|  auto(l4)|   83|
|manual(m6)|   19|
|  auto(s6)|   16|
|  auto(l5)|   39|
|manual(m5)|   58|
|  auto(av)|    5|
|  auto(l3)|    2|
|  auto(l6)|    6|
|  auto(s5)|    3|
|  auto(s4)|    3|
+----------+-----+



In [11]:
# return only auto or manual
# mpg.filter(sqlFunc.col("trans"))
mpg.filter(sqlFunc.col("trans").like("%auto%") | sqlFunc.col("trans").like("%manual%")).show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

### Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [12]:
tips = sp.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [13]:
# get the distinc values of smoker colum
tips.select(tips.smoker).distinct().show()

+------+
|smoker|
+------+
|    No|
|   Yes|
+------+



In [14]:
# get percentage of the value counts
smoker_counts = tips.select(tips.smoker).groupby("smoker").count()
smoker_counts.select(smoker_counts["count"] / tips.count()).show()

+-------------------+
|      (count / 244)|
+-------------------+
| 0.6188524590163934|
|0.38114754098360654|
+-------------------+



### Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [15]:
weather = sp.createDataFrame(vdata("seattle_weather"))
weather.show()

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06 00:00:00|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07 00:00:00|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08 00:00:00|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09 00:00:00|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10 00:00:00|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11 00:00:00|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12 00:00:00|          0.0|     6.1|    -1.7| 1.9|    

In [17]:
# create a month column
weather = weather.withColumn("month",sqlFunc.month("date"))
weather = weather.withColumn("year",sqlFunc.year("date"))
weather = weather.withColumn("day", sqlFunc.dayofmonth("date"))
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+-----+----+---+
|               date|precipitation|temp_max|temp_min|wind|weather|month|year|day|
+-------------------+-------------+--------+--------+----+-------+-----+----+---+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|    1|2012|  1|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|    1|2012|  2|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|    1|2012|  3|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|    1|2012|  4|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|    1|2012|  5|
+-------------------+-------------+--------+--------+----+-------+-----+----+---+
only showing top 5 rows



In [18]:
# Convert the temperatures to fahrenheit.
weather = weather.withColumn("F_temp_max", (weather['temp_max'] * (9/5)) + 32)
weather = weather.withColumn("F_temp_min", (weather['temp_min'] * (9/5)) + 32)
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+-----+----+---+------------------+----------+
|               date|precipitation|temp_max|temp_min|wind|weather|month|year|day|        F_temp_max|F_temp_min|
+-------------------+-------------+--------+--------+----+-------+-----+----+---+------------------+----------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|    1|2012|  1|55.040000000000006|      41.0|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|    1|2012|  2|             51.08|     37.04|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|    1|2012|  3|             53.06|     44.96|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|    1|2012|  4|             53.96|     42.08|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|    1|2012|  5|48.019999999999996|     37.04|
+-------------------+-------------+--------+--------+----+-------+-----+----+---+------------------+----

In [24]:
# Which month has the most rain, on average?
rains = weather.filter(sqlFunc.col("weather").like("%rain%")) # find all row where weater conditions are rain
rains.withColumn("month",sqlFunc.month("date")).groupby("month").count().show()


+-----+-----+
|month|count|
+-----+-----+
|    1|   35|
|    3|   37|
|    5|   16|
|    4|   20|
|    2|   40|
|    6|   19|
|    9|    4|
|    8|    6|
|    7|   14|
|   10|   20|
|   12|   23|
|   11|   25|
+-----+-----+



In [25]:
# Which year was the windiest?
weather.withColumn("year",sqlFunc.year("date")).groupby("year").agg(sqlFunc.sum("precipitation")).show()

+----+------------------+
|year|sum(precipitation)|
+----+------------------+
|2012|1226.0000000000002|
|2013| 828.0000000000002|
|2014|1232.8000000000004|
|2015|1139.1999999999998|
+----+------------------+



In [26]:
# What is the most frequent type of weather in January?
weather.filter(weather["month"] == 1).groupby("weather").count().show()

+-------+-----+
|weather|count|
+-------+-----+
|drizzle|   10|
|   rain|   35|
|    sun|   33|
|   snow|    8|
|    fog|   38|
+-------+-----+



In [27]:
# What is the average high and low temperature on sunny days in July in 2013 and 2014?
weather.filter((weather["year"] == 2013) | (weather["year"] == 2014)\
               & (weather["month"] == 7) & (sqlFunc.col("weather").like("%sun%"))).groupby(\
                                                                                          "year").agg(\
                                                                                      sqlFunc.avg(weather.temp_max), sqlFunc.avg(weather.temp_min)).show()
                                                                                                                                                                                                                    

+----+------------------+------------------+
|year|     avg(temp_max)|     avg(temp_min)|
+----+------------------+------------------+
|2013|16.058904109589037| 8.153972602739726|
|2014|            27.092|14.400000000000002|
+----+------------------+------------------+



In [32]:
# What percentage of days were rainy in q3 of 2015?
weather.filter((weather["year"] == 2015) & (weather["precipitation"] == 0)).groupby("day").count().show()

+---+-----+
|day|count|
+---+-----+
| 31|    5|
| 28|    7|
| 26|    9|
| 12|    6|
| 22|    9|
|  1|    6|
| 13|    5|
|  6|    8|
| 16|    8|
|  3|    8|
| 20|    7|
|  5|    6|
| 15|    8|
| 17|    6|
|  9|    7|
|  4|    9|
|  8|    9|
| 23|    7|
|  7|    6|
| 25|    6|
+---+-----+
only showing top 20 rows



In [33]:
# What percentage of days were rainy in q3 of 2015?
rainny = weather.filter((weather["year"] == 2015) & (\
                                            weather["precipitation"] == 0)).groupby(\
                                                                                    "day").count()

In [38]:
rainny.withColumn("percentage", rainny["count"] / 365).show()

+---+-----+--------------------+
|day|count|          percentage|
+---+-----+--------------------+
| 31|    5|  0.0136986301369863|
| 28|    7|0.019178082191780823|
| 26|    9|0.024657534246575342|
| 12|    6| 0.01643835616438356|
| 22|    9|0.024657534246575342|
|  1|    6| 0.01643835616438356|
| 13|    5|  0.0136986301369863|
|  6|    8|0.021917808219178082|
| 16|    8|0.021917808219178082|
|  3|    8|0.021917808219178082|
| 20|    7|0.019178082191780823|
|  5|    6| 0.01643835616438356|
| 15|    8|0.021917808219178082|
| 17|    6| 0.01643835616438356|
|  9|    7|0.019178082191780823|
|  4|    9|0.024657534246575342|
|  8|    9|0.024657534246575342|
| 23|    7|0.019178082191780823|
|  7|    6| 0.01643835616438356|
| 25|    6| 0.01643835616438356|
+---+-----+--------------------+
only showing top 20 rows

